In [13]:
import pandas as pd 
import torch 
import numpy as np
import torch.nn as nn
import importlib
import pickle
import utils
import models
import dask.array as da

importlib.reload(utils)
from utils import *
importlib.reload(models)
from models import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [14]:
grouped_df = pd.read_csv("../data/2100-jan-jun.csv")
print(grouped_df)

         game_id                                              moves  \
0       1gFKuI4R  g1f3 c7c5 g2g3 g7g6 f1g2 f8g7 e1g1 b8c6 c2c4 d...   
1       6Kp4KfSK  d2d4 d7d5 c2c4 d5c4 b1c3 e7e5 e2e3 b8c6 f1c4 g...   
2       AJ0ErxUz  d2d4 c7c5 g1f3 c5d4 c1f4 b8c6 c2c3 e7e5 f4e5 d...   
3       D1t7sFQ6  d2d4 g8f6 c2c4 e7e6 b1c3 d7d5 g1f3 f8b4 d1b3 b...   
4       M77OjMWp  d2d4 g8f6 c2c4 g7g6 b1c3 f8g7 e2e4 d7d6 f2f4 e...   
...          ...                                                ...   
263267  xpxi2PfS  e2e4 c7c6 d2d4 d7d5 b1c3 d5e4 c3e4 c8f5 e4g3 f...   
263268  y38Wy8Io  d2d4 g8f6 g1f3 g7g6 c1g5 f8g7 b1d2 e8g8 e2e4 d...   
263269  y4GxfsDR  d2d4 g8f6 g1f3 g7g6 c1f4 f8g7 e2e3 d7d6 h2h3 f...   
263270  yANoOutS  c2c4 g8f6 b1c3 e7e6 e2e4 f8b4 e4e5 c7c6 e5f6 b...   
263271  ybpO73Eh  e2e4 e7e5 b1c3 b8c6 f1c4 g8f6 d2d3 f8c5 c1g5 h...   

        white_elo  black_elo  white_active  \
0            2199       2142          True   
1            2104       2177          True   
2        

In [3]:
# Use this when combining memmaps
with open('./../data/black-data/july-sept/vocab.pkl', 'rb') as inp:
    vocab_old = pickle.load(inp)

In [15]:
def save_as_memmap(array, filename):
    # Determine the dtype and shape of the array to create a compatible memmap
    dtype = array.dtype
    shape = array.shape
    
    # Create a memmap file with write mode, which will also allocate the disk space
    memmap_array = np.memmap(filename, dtype=dtype, mode='w+', shape=shape)
    
    # Copy the data into the memmap array
    memmap_array[:] = array[:]
    
    # Flush memory changes to disk to ensure all data is written
    memmap_array.flush()

    # Return the path for confirmation
    return filename

# If you forget the shape
def find_working_shape(filename, dtype, max_first_dim, other_dims):

    # Try decreasing sizes from the max_first_dim until we find a working shape
    for first_dim in range(max_first_dim, 0, -1):
        shape_to_try = (first_dim,) + other_dims
        
        try:
            # Attempt to load the memmap with the current shape
            memmap_array = np.memmap(filename, dtype=dtype, mode='r', shape=shape_to_try)
            # If successful, return the array
            print(f"Successful shape: {shape_to_try}")
            return memmap_array
        except ValueError as e:
            # Catch the ValueError if the shape is not feasible, and try the next size
            continue
    
    raise ValueError("Could not find a working shape within the given bounds.")

# Function to load a memmap file
def load_memmap(filename, dtype, shape):
    # Load the memmap file with read-only mode
    return np.memmap(filename, dtype=dtype, mode='r', shape=shape)

## Transformer Data

In [4]:
trainX, trainY, vocab = df_to_data_fen_only_padded(grouped_df, fixed_window=True, sampling_rate=0.25)

In [5]:
len(vocab.id_to_word)

2524

In [ ]:
print(trainX)

[[3, 6, 4, 0, 0, 0, 0, 0, 0], [3, 31, 4, 0, 0, 0, 0, 0, 0], [3, 35, 36, 37, 4, 0, 0, 0, 0], [3, 43, 37, 4, 0, 0, 0, 0, 0], [3, 58, 4, 0, 0, 0, 0, 0, 0], [3, 26, 4, 0, 0, 0, 0, 0, 0], [3, 69, 37, 4, 0, 0, 0, 0, 0], [3, 42, 4, 0, 0, 0, 0, 0, 0], [3, 6, 4, 0, 0, 0, 0, 0, 0], [3, 77, 4, 0, 0, 0, 0, 0, 0], [3, 81, 4, 0, 0, 0, 0, 0, 0], [3, 85, 4, 0, 0, 0, 0, 0, 0], [3, 91, 37, 4, 0, 0, 0, 0, 0], [3, 92, 37, 4, 0, 0, 0, 0, 0], [3, 61, 37, 4, 0, 0, 0, 0, 0], [3, 94, 37, 4, 0, 0, 0, 0, 0], [3, 97, 37, 4, 0, 0, 0, 0, 0], [3, 98, 4, 0, 0, 0, 0, 0, 0], [3, 106, 4, 0, 0, 0, 0, 0, 0], [3, 107, 37, 4, 0, 0, 0, 0, 0], [3, 53, 4, 0, 0, 0, 0, 0, 0], [3, 52, 4, 0, 0, 0, 0, 0, 0], [3, 46, 4, 0, 0, 0, 0, 0, 0], [3, 57, 4, 0, 0, 0, 0, 0, 0], [3, 114, 4, 0, 0, 0, 0, 0, 0], [3, 54, 4, 0, 0, 0, 0, 0, 0], [3, 115, 4, 0, 0, 0, 0, 0, 0], [3, 48, 4, 0, 0, 0, 0, 0, 0], [3, 118, 37, 4, 0, 0, 0, 0, 0], [3, 64, 4, 0, 0, 0, 0, 0, 0], [3, 123, 36, 37, 4, 0, 0, 0, 0], [3, 128, 4, 0, 0, 0, 0, 0, 0], [3, 132, 36, 37, 4, 0

KeyboardInterrupt: 

In [7]:
trainX, trainY = np.asarray(trainX, dtype=np.int32), np.asarray(trainY, dtype=np.int32)

In [9]:
# for transformer data
filenames = [
    save_as_memmap(trainX, './../data/transformer/jan-march/trainX.memmap'),
    save_as_memmap(trainY, './../data/transformer/jan-march/trainY.memmap'),
]
with open('./../data/transformer/jan-march/vocab.pkl', 'wb') as outp:
    pickle.dump(vocab, outp, pickle.HIGHEST_PROTOCOL)

check for lengths so we're able to load them back

In [10]:
print(trainX.shape)

(6130889, 585)


In [42]:
# For trainX
dtype_trainX = np.int32  # or the correct dtype for your data
shape_trainX = (2161324, 585)  # replace with the correct shape
trainX = load_memmap('./../data/transformer/jan/trainX.memmap', dtype_trainX, shape_trainX)

# For trainY
dtype_trainY = np.int32 # or the correct dtype for your data
shape_trainY = (2161324, 6)  # replace with the correct shape
trainY = load_memmap('./../data/transformer/jan/trainY.memmap', dtype_trainY, shape_trainY)

In [51]:
print(trainX[13])

[ 1  2  7  8  9 10 11  9  8  7 12 13 13 13 13 13 13 13 13 12 14 14 14 14
 14 14 14 14 12 14 14 14 14 14 14 14 14 12 14 14 14 14 14 14 14 14 12 14
 14 14 14 14 14 14 14 12 15 15 15 15 15 15 15 15 12 16 17 18 19 20 18 17
 16  5 21  5 20 19 11 10  5 22  5 23  5 24  3 72  4  2  7  8  9 10 11  9
  8  7 12 13 13 13 13 13 13 13 13 12 14 14 14 14 14 14 14 14 12 14 14 14
 14 14 14 14 14 12 14 14 15 14 14 14 14 14 12 14 14 14 14 14 14 14 14 12
 15 15 14 15 15 15 15 15 12 16 17 18 19 20 18 17 16  5  9  5 20 19 11 10
  5 22  5 23  5 24  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [52]:
print(vocab.get_word(77))

O-O


## for black

In [17]:
trainX_sequences, fens, trainX, trainY, vocab = df_to_data_black(grouped_df, fixed_window=True, sampling_rate=0.25)
trainX_sequences, trainX_seqlengths  = pad_sequences(trainX_sequences)

In [18]:
trainX, trainX_sequences, fens, trainX_seqlengths, trainY = np.asarray(trainX), np.asarray(trainX_sequences), np.asarray(fens), np.asarray(trainX_seqlengths), np.asarray(trainY)

In [19]:
# for black multi-modal
filenames = [
    save_as_memmap(trainX_sequences, './../data/black-2100/trainX_sequences.memmap'),
    save_as_memmap(trainX, './../data/black-2100/trainX.memmap'),
    save_as_memmap(trainY, './../data/black-2100/trainY.memmap'),
    save_as_memmap(trainX_seqlengths, './../data/black-2100/trainX_seqlengths.memmap')
]

df = pd.DataFrame(fens, columns=['fens'])
csv_filename = './../data/black-2100/fens.csv'
df.to_csv(csv_filename, index=False)

with open('./../data/black-2100/vocab.pkl', 'wb') as outp:
    pickle.dump(vocab, outp, pickle.HIGHEST_PROTOCOL)

In [21]:
len(vocab.move_to_id)

5987

In [20]:
print(trainX.shape)

(1733003, 12, 8, 8)


## for white

In [22]:
trainX_sequences, fens, trainX, trainY, vocab = df_to_data(grouped_df, fixed_window=True, sampling_rate=0.25)
trainX_sequences, trainX_seqlengths  = pad_sequences(trainX_sequences)

In [23]:
trainX, trainX_sequences, fens, trainX_seqlengths, trainY = np.asarray(trainX), np.asarray(trainX_sequences), np.asarray(fens), np.asarray(trainX_seqlengths), np.asarray(trainY)

In [24]:
# for black multi-modal
filenames = [
    save_as_memmap(trainX_sequences, './../data/white-2100/trainX_sequences.memmap'),
    save_as_memmap(trainX, './../data/white-2100/trainX.memmap'),
    save_as_memmap(trainY, './../data/white-2100/trainY.memmap'),
    save_as_memmap(trainX_seqlengths, './../data/white-2100/trainX_seqlengths.memmap')
]

df = pd.DataFrame(fens, columns=['fens'])
csv_filename = './../data/white-2100/fens.csv'
df.to_csv(csv_filename, index=False)

with open('./../data/white-2100/vocab.pkl', 'wb') as outp:
    pickle.dump(vocab, outp, pickle.HIGHEST_PROTOCOL)

In [25]:
print(trainX.shape)

(1766908, 12, 8, 8)


## Other

In [ ]:

dtype_trainX = np.int64
max_first_dim = 3038976  # This is your starting point, the upper bound of the first dimension
other_dims = (12, 8, 8)  # The other dimensions of the shape, assumed to be correct

trainX_filename = './../data/apr-jun/trainX.memmap'

# Try to find a working memmap shape
dummy = find_working_shape(trainX_filename, dtype_trainX, max_first_dim, other_dims)

for jan (black)

In [9]:
# Function to load a memmap file
def load_memmap(filename, dtype, shape):
    # Load the memmap file with read-only mode
    return np.memmap(filename, dtype=dtype, mode='r', shape=shape)

#for jan-march

# Assuming you know the dtype and shape, for example:
# For trainX_sequences
dtype_trainX_sequences = np.int64  # or the correct dtype for your data
shape_trainX_sequences = (1977299, 16)  # replace with the correct shape
trainX_sequences = load_memmap('./../data/black-data/jan/trainX_sequences.memmap', dtype_trainX_sequences, shape_trainX_sequences)

# For trainX
dtype_trainX = np.int64  # or the correct dtype for your data
shape_trainX = (1977299, 12, 8, 8)  # replace with the correct shape
trainX = load_memmap('./../data/black-data/jan/trainX.memmap', dtype_trainX, shape_trainX)

# For trainY
dtype_trainY = np.int64  # or the correct dtype for your data
shape_trainY = (1977299,)  # replace with the correct shape
trainY = load_memmap('./../data/black-data/jan/trainY.memmap', dtype_trainY, shape_trainY)

# For trainX_seqlengths
dtype_trainX_seqlengths = np.int64  # or the correct dtype for your data
shape_trainX_seqlengths = (1977299,)  # replace with the correct shape
trainX_seqlengths = load_memmap('./../data/black-data/jan/trainX_seqlengths.memmap', dtype_trainX_seqlengths, shape_trainX_seqlengths)

for jan-march

In [ ]:
# Function to load a memmap file
def load_memmap(filename, dtype, shape):
    # Load the memmap file with read-only mode
    return np.memmap(filename, dtype=dtype, mode='r', shape=shape)

#for jan-march

# Assuming you know the dtype and shape, for example:
# For trainX_sequences
dtype_trainX_sequences = np.int64  # or the correct dtype for your data
shape_trainX_sequences = (3038976, 16)  # replace with the correct shape
trainX_sequences = load_memmap('./../data/jan-march/trainX_sequences.memmap', dtype_trainX_sequences, shape_trainX_sequences)

# For trainX
dtype_trainX = np.int64  # or the correct dtype for your data
shape_trainX = (3038976, 12, 8, 8)  # replace with the correct shape
trainX = load_memmap('./../data/jan-march/trainX.memmap', dtype_trainX, shape_trainX)

# For trainY
dtype_trainY = np.int64  # or the correct dtype for your data
shape_trainY = (3038976,)  # replace with the correct shape
trainY = load_memmap('./../data/jan-march/trainY.memmap', dtype_trainY, shape_trainY)

# For trainX_seqlengths
dtype_trainX_seqlengths = np.int64  # or the correct dtype for your data
shape_trainX_seqlengths = (3038976,)  # replace with the correct shape
trainX_seqlengths = load_memmap('./../data/jan-march/trainX_seqlengths.memmap', dtype_trainX_seqlengths, shape_trainX_seqlengths)

for april-june

In [51]:
# Function to load a memmap file
def load_memmap(filename, dtype, shape):
    # Load the memmap file with read-only mode
    return np.memmap(filename, dtype=dtype, mode='r', shape=shape)

# Assuming you know the dtype and shape, for example:
# For trainX_sequences
dtype_trainX_sequences = np.int64  # or the correct dtype for your data
shape_trainX_sequences = (2780980, 16)  # replace with the correct shape
trainX_sequences = load_memmap('./../data/apr-jun/trainX_sequences.memmap', dtype_trainX_sequences, shape_trainX_sequences)

# For trainX
dtype_trainX = np.int64  # or the correct dtype for your data
shape_trainX = (2780980, 12, 8, 8)  # replace with the correct shape
trainX = load_memmap('./../data/apr-jun/trainX.memmap', dtype_trainX, shape_trainX)

# For trainY
dtype_trainY = np.int64  # or the correct dtype for your data
shape_trainY = (2780980,)  # replace with the correct shape
trainY = load_memmap('./../data/apr-jun/trainY.memmap', dtype_trainY, shape_trainY)

# For trainX_seqlengths
dtype_trainX_seqlengths = np.int64  # or the correct dtype for your data
shape_trainX_seqlengths = (2780980,)  # replace with the correct shape
trainX_seqlengths = load_memmap('./../data/apr-jun/trainX_seqlengths.memmap', dtype_trainX_seqlengths, shape_trainX_seqlengths)

## let's combine memmaps

In [3]:
def concatenate_memmaps(file1, shape1, file2, shape2, dtype, result_file):
    # Calculate the new shape
    new_shape = (shape1[0] + shape2[0],) + shape1[1:]
    
    # Create a new memmap for the concatenated data
    concatenated = np.memmap(result_file, dtype=dtype, mode='w+', shape=new_shape)
    
    # Load the original memmaps
    memmap1 = np.memmap(file1, dtype=dtype, mode='r', shape=shape1)
    memmap2 = np.memmap(file2, dtype=dtype, mode='r', shape=shape2)
    
    # Copy data from the original memmaps to the new memmap
    concatenated[:shape1[0]] = memmap1[:]
    concatenated[shape1[0]:] = memmap2[:]
    
    # Flush changes to ensure they're written to disk
    concatenated.flush()
    
    return concatenated



In [3]:
# Example usage for trainX_sequences
dtype = np.int64
shape_jan_mar = (3038976, 16)
shape_apr_jun = (2780980, 16)
result_file_sequences = './../data/combined/trainX_sequences.memmap'

concatenated_sequences = concatenate_memmaps(
    './../data/jan-march/trainX_sequences.memmap', shape_jan_mar,
    './../data/apr-jun/trainX_sequences.memmap', shape_apr_jun,
    dtype, result_file_sequences)


# trainX concatenation
shape_jan_mar_trainX = (3038976, 12, 8, 8)
shape_apr_jun_trainX = (2780980, 12, 8, 8)
result_file_trainX = './../data/combined/trainX.memmap'

concatenated_trainX = concatenate_memmaps(
    './../data/jan-march/trainX.memmap', shape_jan_mar_trainX,
    './../data/apr-jun/trainX.memmap', shape_apr_jun_trainX,
    dtype_trainX, result_file_trainX)

# trainY concatenation
shape_jan_mar_trainY = (3038976,)
shape_apr_jun_trainY = (2780980,)
result_file_trainY = './../data/combined/trainY.memmap'

concatenated_trainY = concatenate_memmaps(
    './../data/jan-march/trainY.memmap', shape_jan_mar_trainY,
    './../data/apr-jun/trainY.memmap', shape_apr_jun_trainY,
    dtype_trainY, result_file_trainY)

# trainX_seqlengths concatenation
shape_jan_mar_seqlengths = (3038976,)
shape_apr_jun_seqlengths = (2780980,)
result_file_seqlengths = './../data/combined/trainX_seqlengths.memmap'

concatenated_seqlengths = concatenate_memmaps(
    './../data/jan-march/trainX_seqlengths.memmap', shape_jan_mar_seqlengths,
    './../data/apr-jun/trainX_seqlengths.memmap', shape_apr_jun_seqlengths,
    dtype_trainX_seqlengths, result_file_seqlengths)

FileNotFoundError: [Errno 2] No such file or directory: './../data/jan-march/trainX_sequences.memmap'

let's do it for black data

In [9]:
def concatenate_memmaps(files, shapes, dtype, result_file):
    if len(files) != len(shapes):
        raise ValueError("The number of files must match the number of shapes")

    # Calculate the total number of rows for the new shape
    total_rows = sum(shape[0] for shape in shapes)
    # Assume all shapes have the same dimensions beyond the first axis
    new_shape = (total_rows,) + shapes[0][1:]
    
    # Create a new memmap for the concatenated data
    concatenated = np.memmap(result_file, dtype=dtype, mode='w+', shape=new_shape)
    
    # Initial index for data insertion in the concatenated memmap
    current_index = 0
    
    # Loop over each file and corresponding shape
    for file, shape in zip(files, shapes):
        print(file)
        # Load the original memmap
        memmap = np.memmap(file, dtype=dtype, mode='r', shape=shape)
        
        # Copy data to the appropriate slice of the concatenated memmap
        concatenated[current_index:current_index + shape[0]] = memmap
        
        # Update the index for the next insertion point
        current_index += shape[0]
    
    # Flush changes to ensure they're written to disk
    concatenated.flush()

def concatenate_memmaps_list(files, shapes, dtype, result_file):
    if len(files) != len(shapes):
        raise ValueError("The number of files must match the number of shapes")
    # Loop over each file and corresponding shape
    memmaps = []
    for file, shape in zip(files, shapes):
        print(file)
        # Load the original memmap
        memmap = np.memmap(file, dtype=dtype, mode='r', shape=shape)
        memmaps.append(memmap)
    
    # Flush changes to ensure they're written to disk
    # concatenated.flush()
    
    return da.concatenate(memmaps,axis=0)

In [10]:
trainX_files = ['./../data/black-data/jan/trainX.memmap',
                './../data/black-data/feb/trainX.memmap', 
                './../data/black-data/march/trainX.memmap',
                './../data/black-data/apr-jun/trainX.memmap',
                './../data/black-data/july-sept/trainX.memmap',
                './../data/black-data/oct-dec/trainX.memmap'
                ]

trainY_files = ['./../data/black-data/jan/trainY.memmap',
                './../data/black-data/feb/trainY.memmap',
                './../data/black-data/march/trainY.memmap',
                './../data/black-data/apr-jun/trainY.memmap',
                './../data/black-data/july-sept/trainY.memmap',
                './../data/black-data/oct-dec/trainY.memmap'
               ]

trainX_sequences_files = ['./../data/black-data/jan/trainX_sequences.memmap',
                          './../data/black-data/feb/trainX_sequences.memmap',
                          './../data/black-data/march/trainX_sequences.memmap',
                          './../data/black-data/apr-jun/trainX_sequences.memmap',
                          './../data/black-data/july-sept/trainX_sequences.memmap',
                          './../data/black-data/oct-dec/trainX_sequences.memmap'
                         ]

trainX_seqlengths_files = ['./../data/black-data/jan/trainX_seqlengths.memmap',
                           './../data/black-data/feb/trainX_seqlengths.memmap',
                           './../data/black-data/march/trainX_seqlengths.memmap',
                           './../data/black-data/apr-jun/trainX_seqlengths.memmap',
                           './../data/black-data/july-sept/trainX_seqlengths.memmap',
                           './../data/black-data/oct-dec/trainX_seqlengths.memmap'
                          ]

trainX_shapes = [(1071641, 12, 8, 8),
                 (922280, 12, 8, 8),
                 (1040912, 12, 8, 8),
                 (2726090, 12, 8, 8),
                 (2654080, 12, 8, 8),
                 (2965521, 12, 8, 8)
                 ]

train_shapes = [(1071641,),
                 (922280,),
                 (1040912,),
                 (2726090,),
                 (2654080,),
                 (2965521,)
                 ]

sequence_shapes =[(1071641,16),
                 (922280,16),
                 (1040912,16),
                 (2726090,16),
                 (2654080,16),
                 (2965521,16)
                 ]

concatenate_memmaps(trainX_files,trainX_shapes,np.int64, './../data/black-data/combined/trainX.memmap')

./../data/black-data/jan/trainX.memmap
./../data/black-data/feb/trainX.memmap
./../data/black-data/march/trainX.memmap
./../data/black-data/apr-jun/trainX.memmap
./../data/black-data/july-sept/trainX.memmap
./../data/black-data/oct-dec/trainX.memmap


In [11]:
concatenate_memmaps(trainY_files,train_shapes,np.int64, './../data/black-data/combined/trainY.memmap')
concatenate_memmaps(trainX_sequences_files,sequence_shapes,np.int64, './../data/black-data/combined/trainX_sequences.memmap')
concatenate_memmaps(trainX_seqlengths_files,train_shapes,np.int64, './../data/black-data/combined/trainX_seqlengths.memmap')

./../data/black-data/jan/trainY.memmap
./../data/black-data/feb/trainY.memmap
./../data/black-data/march/trainY.memmap
./../data/black-data/apr-jun/trainY.memmap
./../data/black-data/july-sept/trainY.memmap
./../data/black-data/oct-dec/trainY.memmap
./../data/black-data/jan/trainX_sequences.memmap
./../data/black-data/feb/trainX_sequences.memmap
./../data/black-data/march/trainX_sequences.memmap
./../data/black-data/apr-jun/trainX_sequences.memmap
./../data/black-data/july-sept/trainX_sequences.memmap
./../data/black-data/oct-dec/trainX_sequences.memmap
./../data/black-data/jan/trainX_seqlengths.memmap
./../data/black-data/feb/trainX_seqlengths.memmap
./../data/black-data/march/trainX_seqlengths.memmap
./../data/black-data/apr-jun/trainX_seqlengths.memmap
./../data/black-data/july-sept/trainX_seqlengths.memmap
./../data/black-data/oct-dec/trainX_seqlengths.memmap


In [ ]:
# Function to load a memmap file
def load_memmap(filename, dtype, shape):
    # Load the memmap file with read-only mode
    return np.memmap(filename, dtype=dtype, mode='r', shape=shape)

# Assuming you know the dtype and shape, for example:
# For trainX_sequences
dtype_trainX_sequences = np.int64  # or the correct dtype for your data
shape_trainX_sequences = (11380524, 16)  # replace with the correct shape
trainX_sequences = load_memmap('./../data/black-data/combined/trainX_sequences.memmap', dtype_trainX_sequences, shape_trainX_sequences)

# For trainX
dtype_trainX = np.int64  # or the correct dtype for your data
shape_trainX = (11380524, 12, 8, 8)  # replace with the correct shape
trainX = load_memmap('./../data/black-data/combined/trainX.memmap', dtype_trainX, shape_trainX)

# For trainY
dtype_trainY = np.int64  # or the correct dtype for your data
shape_trainY = (11380524,)  # replace with the correct shape
trainY = load_memmap('./../data/black-data/combined/trainY.memmap', dtype_trainY, shape_trainY)

# For trainX_seqlengths
dtype_trainX_seqlengths = np.int64  # or the correct dtype for your data
shape_trainX_seqlengths = (11380524,)  # replace with the correct shape
trainX_seqlengths = load_memmap('./../data/black-data/combined/trainX_seqlengths.memmap', dtype_trainX_seqlengths, shape_trainX_seqlengths)